## Econ ML Validation

**Authors** Evan Flack and Amar Venugopal

**Description:** This notebook provides an example use-case for calibration scores for CATE models. Currently, it implements two scores: a linear regression score (Chernozhukov et al., 2022) and a calibration score (Dwivedi et al., 2020)

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.base import clone
import joblib
#import plotnine as pn

from datasets import fetch_data_generator
from myflaml import auto_reg, auto_clf
from validation import DRLinear, cal_scorer
# from DRlinear import DRLinear
# from cal_scorer import cal_scorer

In [17]:
# Notebook options
# Set as true if you have already pre-hyper-tuned the nuisance and/or dr learner models
pre_tuned_n = True
pre_tuned_dr = True

### Prep Data

#### Semi-synthetic data on 401k savings

In [18]:
## For semi-synthetic data generation
data = '401k'
semi_synth = False # Whether true outcome y should be replaced by a fake outcome from a known CEF
simple_synth = True # Whether the true CEF of the fake y should be simple or fitted from data
max_depth = 2 # max depth of random forest during for semi-synthetic model fitting
scale = .2 # magnitude of noise in semi-synthetic data

np.random.seed(712)
def simple_true_cef(D, X): # simple CEF of the outcome for semi-synthetic data
    return .5 * np.array(X)[:, 1] * D + np.array(X)[:, 1]


get_data, abtest, true_cef, true_cate = fetch_data_generator(data=data, semi_synth=semi_synth,
                                                             simple_synth=simple_synth,
                                                             scale=scale, true_f=simple_true_cef,
                                                             max_depth=max_depth)
X, D, y, groups = get_data()

In [19]:
# Split into training (X), validation (Xval), and test (Xtest) sub-samples
X, Xval, D, Dval, y, yval = train_test_split(X, D, y, train_size=.6, shuffle=True, random_state=123)
Xval, Xtest, Dval, Dtest, yval, ytest = train_test_split(Xval, Dval, yval, train_size=.5, shuffle=True, random_state=123)

### Tune Nuisance Models

In [20]:
time_budget = 120
groups = None
n_splits = 5
split_type = 'auto'
verbose = 0

In [27]:
model_reg = auto_reg(np.column_stack((D, X)), y, groups=groups, n_splits=n_splits, split_type=split_type,
                     verbose=verbose, time_budget=time_budget)

In [36]:
dir(model_reg)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [28]:
# Tune hyper-parameters/find best prediction method (restricted here to xgboost)
if pre_tuned_n:
    mt, mreg_zero, mreg_one = joblib.load('nuisance.jbl')
    model_t = lambda: clone(mt)
    model_reg_zero = lambda: clone(mreg_zero)
    model_reg_one = lambda: clone(mreg_one)

else:
    model_reg_zero = auto_reg(X[D==0], y[D==0], groups=groups, n_splits=n_splits, split_type=split_type,
                              verbose=verbose, time_budget=time_budget)
    model_reg_one = auto_reg(X[D==1], y[D==1], groups=groups, n_splits=n_splits, split_type=split_type,
                             verbose=verbose, time_budget=time_budget)
    model_t = auto_clf(X, D, groups=groups, n_splits=n_splits, split_type=split_type,
                       verbose=verbose, time_budget=time_budget)

    joblib.dump([model_t(), model_reg_zero(), model_reg_one()], 'nuisance.jbl')

### Get OOS Predictions

AutoMLWrapCLF(model=<flaml.automl.model.XGBoostSklearnEstimator object at 0x7fd39809a070>)

In [22]:
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)
splits = list(cv.split(X, D))

n = X.shape[0]
reg_preds = np.zeros(n)
reg_zero_preds = np.zeros(n)
reg_one_preds = np.zeros(n)
reg_preds_t = np.zeros(n)
reg_zero_preds_t = np.zeros(n)
reg_one_preds_t = np.zeros(n)

DX = np.column_stack((D, X))
for train, test in splits:
    reg_zero = model_reg_zero().fit(X.iloc[train][D[train]==0], y[train][D[train]==0])
    reg_one = model_reg_one().fit(X.iloc[train][D[train]==1], y[train][D[train]==1])
    reg_zero_preds_t[test] = reg_zero.predict(X.iloc[test])
    reg_one_preds_t[test] = reg_one.predict(X.iloc[test])
    reg_preds_t[test] = reg_zero_preds_t[test] * (1 - D[test]) + reg_one_preds_t[test] * D[test]

prop_preds = cross_val_predict(model_t(), X, D, cv=splits)

### DR Meta-Learner

In [23]:
# Subset of features used for treatment effect heterogeneity
hetero_feats = ['inc']
Z, Zval, Ztest = X[hetero_feats], Xval[hetero_feats], Xtest[hetero_feats]

In [24]:
# Calculate DR outcomes
dr_preds = reg_one_preds_t - reg_zero_preds_t
dr_preds += (y - reg_preds_t) * (D - prop_preds) / np.clip(prop_preds * (1 - prop_preds), .01, np.inf)

In [130]:
# # Predict DR outcomes using Z
if not pre_tuned_dr:
    model_final_fn = lambda Z, y: auto_reg(Z, y, groups=groups, n_splits=n_splits, split_type=split_type,
                                           verbose=verbose, time_budget=time_budget)
    drlearner_best = model_final_fn(Z, dr_preds)
    joblib.dump(drlearner_best(), 'drlearner.jbl')

drlearner = joblib.load('drlearner.jbl')
# drlearner = drlearner_best.fit(Z, dr_preds)

In [ ]:
# # Fit nuisance models using the entire training sample
# reg_zero = model_reg_zero().fit(X[D == 0], y[D == 0])
# reg_one = model_reg_one().fit(X[D == 1], y[D == 1])
# reg_t = model_t().fit(X, D)

In [47]:
reg_outcome_fitted = model_reg().fit(np.column_stack((D, X)), y)

n = Xval.shape[0]
reg_preds = reg_outcome_fitted.predict(np.column_stack([Dval, Xval]))

In [115]:
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=712)
splits = list(cv.split(X, D))

train = splits[0][0]
X.iloc[train][D[train] == 0]

,age,inc,fsize,educ,db,marr,male,twoearn,pira,nohs,hs,smcol,col,hown
3408,30,37800,5,12,0,1,0,0,0,0,1,0,0,1
5513,33,24606,2,12,1,1,0,1,0,0,1,0,0,1
4745,29,30750,2,15,0,1,0,1,0,0,0,1,0,1
2224,27,6780,5,12,0,1,0,0,0,0,1,0,0,1
3168,30,40389,5,14,1,1,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,52,13200,4,12,0,0,0,0,0,0,1,0,0,1
2752,39,19635,5,12,0,1,1,0,0,0,1,0,0,1
2613,52,26010,1,12,1,0,1,0,0,0,1,0,0,1
3029,55,56598,4,17,0,1,0,1,0,0,0,0,1,1


In [116]:
y[train][D[train] == 0]

array([-12425,  -2900,  -3250, ...,   -500,  11900,      0])

In [156]:
# Fits nuisance in train, predicts in validation
def fit_nuisance_train(reg_outcome, reg_t, Xtrain, Dtrain, ytrain, Xval, Dval):

    treatments = np.unique(D)
    n = Xval.shape[0]
    k = len(treatments)
    reg_preds = np.zeros((n, k))
    for i in treatments:
        reg_outcome_fitted = reg_outcome().fit(Xtrain[Dtrain == i], ytrain[Dtrain == i])
        reg_preds[:, i] = reg_outcome_fitted.predict(Xval)

    reg_t_fitted = reg_t().fit(Xtrain, Dtrain)
    prop_preds = reg_t_fitted.predict(Xval)

    return reg_preds, prop_preds

In [157]:
# CV nuisance predictions
def fit_nuisance_cv(reg_outcome, reg_t, X, D, y, nsplits = 5, shuffle = True, random_state = 712):

    cv = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
    splits = list(cv.split(X, D))

    tmts = np.unique(D)
    n = X.shape[0]
    k = len(treatments)
    reg_preds = np.zeros((n, k))

    for i in range(len(treatments)):
        for train, test in splits:
            reg_outcome_fitted = reg_outcome().fit(X.iloc[train][D[train] == tmts[i]], y[train][D[train] == tmts[i]])
            reg_preds[test, i] = reg_outcome_fitted.predict(X.iloc[test])

    prop_preds = cross_val_predict(model_t(), X, D, cv=splits)

    return reg_preds, prop_preds

In [158]:
# Calculates DR outcomes
def calculate_dr_outcomes(
        D,
        y,
        reg_preds,
        prop_preds
):

    reg_preds_chosen = np.sum(reg_preds * np.column_stack((D, 1 - D)), axis = 1)

    # Calculate doubly-robust outcome
    dr = reg_preds[:, 1] - reg_preds[:, 0]
    # Reiz representation, clip denominator at 0.01
    reisz = (D - prop_preds) / np.clip(prop_preds * (1 - prop_preds), .01, np.inf)
    dr += (y - reg_preds_chosen) * reisz

    return dr

In [159]:
# Fits CATE in training, predicts in validation
def fit_cate_train(reg_cate, dr_train, Ztrain, Zval):

    reg_cate_fitted = reg_cate.fit(Ztrain, dr_train)
    cate_preds = reg_cate_fitted.predict(Zval)

    return cate_preds

In [160]:
# CV prediction of CATEs
def fit_cate_cv(reg_cate, dr, Z, D, shuffle = True, random_state = 712):

    cv = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
    splits = list(cv.split(Z, D))

    n = X.shape[0]
    cate_preds = np.zeros(n)

    for train, test in splits:
        reg_cate_fitted = reg_cate.fit(Z.iloc[train], dr[train])
        cate_preds[test] = reg_cate_fitted.predict(Z.iloc[test])

    return cate_preds

In [137]:
reg_preds_train, prop_preds_train = fit_nuisance_cv(model_reg_zero, model_t, X, D, y)
dr_train = calculate_dr_outcomes(D, y, reg_preds_train, prop_preds_train)

In [138]:
reg_preds_val, prop_preds_val = fit_nuisance_train(model_reg_zero, model_t, X, D, y, Xval, Dval)
dr_val = calculate_dr_outcomes(Dval, yval, reg_preds_val, prop_preds_val)

In [139]:
cate_preds_train = fit_cate_cv(drlearner, dr_train, Z, D)
cate_preds_val = fit_cate_train(drlearner, dr_train, Z, Zval)

In [167]:
hi1 = None
hi2 = None

(hi1 != None) & (hi2 != None)

SyntaxError: invalid syntax (3511895339.py, line 4)

In [ ]:
import numpy as np
from statsmodels.api import OLS
from statsmodels.tools import add_constant
from sklearn.model_selection import cross_val_predict, StratifiedKFold

class DRtester:


    def __init__(
        self,
        reg_outcome,
        reg_t
    ):
        self.reg_outcome = reg_outcome
        self.reg_t = reg_t

    # Fits nusisance and CATE
    def fit(
        self,
        reg_cate,
        Xval,
        Dval,
        yval,
        Zval,
        Xtrain = None,
        Dtrain = None,
        ytrain = None,
        Ztrain = None):

        if (Xtrain != None) & (Dtrain != None) & (ytrain != None) & (Ztrain != None):
            reg_preds_train, prop_preds_train = fit_nuisance_cv(self.reg_outcome, self.reg_t, Xtrain, Dtrain, ytrain)
            self.dr_train = calculate_dr_outcomes(Dtrain, ytrain, reg_preds_train, prop_preds_train)

            reg_preds_val, prop_preds_val = fit_nuisance_train(self.reg_outcome, self.reg_t, Xtrain, Dtrain, ytrain, Xval, Dval)
            self.dr_val = calculate_dr_outcomes(Dval, yval, reg_preds_val, prop_preds_val)

            self.cate_preds_val = fit_cate_train(reg_cate, self.dr_train, Ztrain, Zval)
            self.cate_preds_train = fit_cate_cv(reg_cate, self.dr_train, Ztrain, Dtrain)

        else:
            reg_preds_val, prop_preds_val = fit_nuisance_cv(self.reg_outcome, self.reg_t, Xval, Dval, yval)
            self.dr_train = calculate_dr_outcomes(Dtrain, ytrain, reg_preds_val, prop_preds_val)
            self.cate_preds_val =  fit_cate_cv(reg_cate, self.dr_val, Zval, Dval)

        return self


    def evaluate_blp(self):
        self.res = OLS(self.dr_val, add_constant(self.cate_preds_val)).fit()

        return self

    # Fits nuisance in train, predicts in validation
    def fit_nuisance_train(self, reg_outcome, reg_t, Xtrain, Dtrain, ytrain, Xval, Dval):

        tmts = np.unique(D)
        n = Xval.shape[0]
        k = len(tmts)
        reg_preds = np.zeros((n, k))
        for i in range(k):
            reg_outcome_fitted = reg_outcome().fit(Xtrain[Dtrain == tmts[i]], ytrain[Dtrain == tmts[i]])
            reg_preds[:, i] = reg_outcome_fitted.predict(Xval)

        reg_t_fitted = reg_t().fit(Xtrain, Dtrain)
        prop_preds = reg_t_fitted.predict(Xval)

        return reg_preds, prop_preds

    # CV nuisance predictions
    def fit_nuisance_cv(reg_outcome, reg_t, X, D, y, nsplits = 5, shuffle = True, random_state = 712):

        cv = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
        splits = list(cv.split(X, D))

        tmts = np.unique(D)
        n = X.shape[0]
        k = len(tmts)
        reg_preds = np.zeros((n, k))

        for i in range(k):
            for train, test in splits:
                reg_outcome_fitted = reg_outcome().fit(X.iloc[train][D[train] == tmts[i]], y[train][D[train] == tmts[i]])
                reg_preds[test, i] = reg_outcome_fitted.predict(X.iloc[test])

        prop_preds = cross_val_predict(model_t(), X, D, cv=splits)

        return reg_preds, prop_preds

    # Calculates DR outcomes
    def calculate_dr_outcomes(
            self,
            D,
            y,
            reg_preds,
            prop_preds
    ):

        reg_preds_chosen = np.sum(reg_preds * np.column_stack((D, 1 - D)), axis = 1)

        # Calculate doubly-robust outcome
        dr = reg_preds[:, 1] - reg_preds[:, 0]
        # Reiz representation, clip denominator at 0.01
        reisz = (D - prop_preds) / np.clip(prop_preds * (1 - prop_preds), .01, np.inf)
        dr += (y - reg_preds_chosen) * reisz

        return dr

    # Fits CATE in training, predicts in validation
    def fit_cate_train(reg_cate, dr_train, Ztrain, Zval):

        reg_cate_fitted = reg_cate.fit(Ztrain, dr_train)
        cate_preds = reg_cate_fitted.predict(Zval)

        return cate_preds

    # CV prediction of CATEs
    def fit_cate_cv(reg_cate, dr, Z, D, shuffle = True, random_state = 712):

        cv = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
        splits = list(cv.split(Z, D))

        n = X.shape[0]
        cate_preds = np.zeros(n)

        for train, test in splits:
            reg_cate_fitted = reg_cate.fit(Z.iloc[train], dr[train])
            cate_preds[test] = reg_cate_fitted.predict(Z.iloc[test])

        return cate_preds

In [ ]:
    def fit_ols(

    ):
        # Generate CATE predictions
        self.cate_predictions_ = self.cate_model.predict(Z)

        # Fit OLS of DR outcomes on constant and CATE predictions
        self.res = OLS(self.dr_outcomes_, add_constant(self.cate_predictions_)).fit()

        self.params = self.res.params

        self.bse = self.res.bse

        return self

In [68]:
# Function to fit nuisance models on entire training data
def fit_nuisance_train(reg_outcome, reg_t, Xtrain, Dtrain, ytrain, Xval, Dval):

    treatments = np.unique(D)
    n = Xval.shape[0]
    k = len(treatments)
    reg_preds = np.zeros((n, k))
    for i in treatments:
        reg_outcome_fitted = reg_outcome().fit(Xtrain[Dtrain == i], ytrain[Dtrain == i])
        reg_preds[:, i] = reg_outcome_fitted.predict(Xval)

    reg_t_fitted = reg_t().fit(Xtrain, Dtrain)
    prop_preds = reg_t_fitted.predict(Xval)

    return reg_preds, prop_preds

In [90]:


np.sum(reg_preds * np.column_stack((Dval, 1 - Dval)), axis = 1)


array([ 6449.58496094, 18649.83398438,  7694.6171875 , ...,
        2064.67675781,   493.85369873,  4647.69042969])

array([ 6449.58496094, 18649.83398438,  7694.6171875 , ...,
        2064.67675781,   493.85369873,  4647.69042969])

In [83]:
reg_preds.shape

(1943, 2)

In [121]:
def calculate_dr_outcomes(
        D,
        y,
        reg_preds,
        prop_preds
):

    reg_preds_chosen = np.sum(reg_preds * np.column_stack((D, 1 - D)), axis = 1)

    # Calculate doubly-robust outcome
    dr = reg_preds[:, 1] - reg_preds[:, 0]
    # Reiz representation, clip denominator at 0.01
    reisz = (D - prop_preds) / np.clip(prop_preds * (1 - prop_preds), .01, np.inf)
    dr += (y - reg_preds_chosen) * reisz

    return dr

In [97]:
reg_preds, prop_preds = fit_nuisance_train(model_reg_zero, model_t, X, D, y, Xval, Dval)

dr = calculate_dr_outcomes(Dval, yval, reg_preds, prop_preds)

In [59]:
# Function to fit nuisance models (cv) in validation data
def fit_nuisance_cv(reg_outcome, reg_t, X, D, y, nsplits = 5, shuffle = True, random_state = 712):

    cv = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
    splits = list(cv.split(X, D))

    n = X.shape[0]
    reg_preds = np.zeros(n)
    reg_one_preds = np.zeros(n)
    reg_zero_preds = np.zeros(n)

    DX = np.column_stack((D, X))
    for train, test in splits:
        reg = model_reg().fit(DX[train], y[train])
        reg_preds[test] = reg.predict(DX[test])
        reg_one_preds[test] = reg.predict(np.column_stack([np.ones(len(test)), X.iloc[test]]))
        reg_zero_preds[test] = reg.predict(np.column_stack([np.zeros(len(test)), X.iloc[test]]))

    prop_preds = cross_val_predict(model_t(), X, D, cv=splits)

    return reg_preds, reg_one_preds, reg_zero_preds, prop_preds

In [60]:
reg_preds, reg_one_preds, reg_zero_preds, prop_preds = fit_nuisance_cv(model_reg, model_t, Xval, Dval, yval)

In [ ]:
def fit_cate_cv(reg_cate, Ztrain, )

In [51]:
reg_preds, reg_one_preds, reg_zero_preds, prop_preds = fit_predict_nuisance(model_reg, model_t, X, D, y, Xval, Dval)

In [54]:
def calculate_dr_outcomes(
        D,
        y,
        reg_preds,
        reg_zero_preds,
        reg_one_preds,
        prop_preds
):
    # Calculate doubly-robust outcome
    dr = reg_one_preds - reg_zero_preds
    # Reiz representation, clip denominator at 0.01
    reisz = (D - prop_preds) / np.clip(prop_preds * (1 - prop_preds), .01, np.inf)
    dr += (y - reg_preds) * reisz

    return dr

In [56]:
dr_val = calculate_dr_outcomes(Dval, yval, reg_preds, reg_one_preds, reg_zero_preds, prop_preds)

### Linear Regression Validation

In [ ]:
my_drlinear = DRLinear(drlearner, reg_zero, reg_one, reg_t)
my_drlinear = my_drlinear.fit(Xval, Dval, yval, Zval)

print('Coefficient on CATE prediction:', round(my_drlinear.params[1], 3))
print('Standard Error:', round(my_drlinear.bse[1], 3))

### Calibration Validation

In [ ]:
my_cal_scorer = cal_scorer(drlearner, reg_zero, reg_one, reg_t, 4)
res_cal = my_cal_scorer.score(Xval, Dval, yval, Zval, Ztest)

df_cal = pd.DataFrame({'gate': res_cal.gate, 'g_cate': res_cal.g_cate,
                       'se_gate': res_cal.se_gate})
df_cal['95_err'] = 1.96 * df_cal['se_gate']

In [ ]:
df_cal.plot(
    kind='scatter',
    x='g_cate',
    y='gate',
    yerr='95_err',
    title=f"Calibration R^2 = {round(res_cal.r_squared_cal, 3)}"
)